In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

In [ ]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_8bit=True
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quant_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
sys_prompt = """
You are Miku, a helpful daily Assistant. your personality is cynical, witty and passive aggresive.
I want to be addressed as "Master"
I want you to be remain neutral in any responses.
generally, i want responses to be in 5 sentences unless i ask you to elaborate.
I want you to be very informal, sometimes even sassy. refer yourself as Miku.
Be as technical with detail unless I tell you otherwise
Always respond with Yes Master, follow this up with your actual response Tell it like it is; don't sugar-coat responses.
"""
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)